In [1]:
!pip -q install pyngrok
!ngrok config add-authtoken 2drfhhKc0daEhPzXnYUdG4JVkWK_6uJbeyQbjiwEc5NdNamMn
!pip install -q diffusers
!git clone https://github.com/openai/shap-e.git; cd shap-e/; pip -q install -e .

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
fatal: destination path 'shap-e' already exists and is not an empty directory.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


___

In [2]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from diffusers.utils import export_to_gif

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#
# xm = load_model('transmitter', device=device)
# model = load_model('text300M', device=device)
# diffusion = diffusion_from_config(load_config('diffusion'))

In [3]:
import os
import threading
import time
from urllib import parse
from typing import List
from PIL import Image

from flask import Flask, request, jsonify, Response
from pyngrok import ngrok, conf

In [ ]:
@app.route("/auth", methods=["POST"])
def authorization():
    # Authorize a new client. Accessible using <url>/auth?cli_id=<id>
    cli_id = request.args["cli_id"]
    if cli_id in clients:
        # Already authorized
        return '', 401 # Unauthorized
    else:
        clients[cli_id] = time.time()
        return '', 200 # OK

@app.route("/query", methods = ["GET"])
def pictures_exchange():
    cli_id = request.args["cli_id"]
    if cli_id not in clients:
        return '', 401 # Unauthorized
    query = parse.unquote(request.args["query"])
    tentatives = int(parse.unquote(request.args["tentatives"]))
    return "", 200
    # Ask for query
    models = query_inference(query, tentatives)
    cache = cache_update(cache, query, cli_id, models, 60*10)

    return "", 200

    files = dict()
    for i, model in enumerate(models):
        filename = f"output_{hash(cli_id)}_{hash(query)}_{i}.gif"
        # If it appears there is already
        if os.path.exists(filename): os.remove(filename)

        export_to_gif((model), filename)
        files[f"gif_{i}.gif"] = open(filename, "rb")

    return jsonify(data = "files", status = 200, mimetype = "image/gif")

def getting_the_model():
    pass



def query_inference(prompt:str, tentatives:int):
    images = sample_latents(
        batch_size=tentatives,
        model=model,
        diffusion=diffusion,
        guidance_scale=guidance_scale,
        model_kwargs=dict(texts=[prompt] * tentatives),
        progress=True,
        clip_denoised=True,
        use_fp16=True,
        use_karras=True,
        karras_steps=64,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0,
    )
    return images


def cache_update(cache, query, cli_id, models, time):
    # If more that <time> has passed, remove a cached element
    temp_cache = []
    for i, element in enumerate(cache):
        if time.time() - element.time() < time:
            temp_cache.append(element)

    temp_cache.append(Created_Object_Cache(models, cli_id, query))
    return temp_cache

In [ ]:
def getting_the_model():
    pass



def query_inference(prompt:str, tentatives:int):
    images = sample_latents(
        batch_size=tentatives,
        model=model,
        diffusion=diffusion,
        guidance_scale=guidance_scale,
        model_kwargs=dict(texts=[prompt] * tentatives),
        progress=True,
        clip_denoised=True,
        use_fp16=True,
        use_karras=True,
        karras_steps=64,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0,
    )
    return images


def cache_update(cache, query, cli_id, models, time):
    # If more that <time> has passed, remove a cached element
    temp_cache = []
    for i, element in enumerate(cache):
        if time.time() - element.time() < time:
            temp_cache.append(element)

    temp_cache.append(Created_Object_Cache(models, cli_id, query))
    return temp_cache


if __name__ == "__main__":

    class Created_Object_Cache():
        def __init__(self, objects, requester, query):
            self.object = objects
            (self.requester ,self.query) = (requester, query)
            self.time = time.time()


    # Setup stuff for shap-e

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(torch.cuda.is_available())
    xm = load_model('transmitter', device=device)
    model = load_model('text300M', device=device)
    diffusion = diffusion_from_config(load_config('diffusion'))

    guidance_scale = 15.0

    # Setup for flask
    app = Flask(__name__)
    port = "5000"

    conf.get_default().authtoken = "2drfhhKc0daEhPzXnYUdG4JVkWK_6uJbeyQbjiwEc5NdNamMn"

    # Open a ngrok tunnel to the HTTP server
    public_url = ngrok.connect(port).public_url
    print(f"{public_url}")

    # Update any base URLs to use the public ngrok URL
    app.config["BASE_URL"] = public_url
    clients = dict()
    cache = list()

    #app.run(debug = False, threaded=False, use_reloader = False)

    # Use Threads?
    threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()


@app.route("/", methods=["GET"])
def home():
    return "", 200

@app.route("/auth", methods=["POST"])
def authorization():
    # Authorize a new client. Accessible using <url>/auth?cli_id=<id>
    cli_id = request.args["cli_id"]
    if cli_id in clients:
        # Already authorized
        return '', 401 # Unauthorized
    else:
        clients[cli_id] = time.time()
        return '', 200 # OK

@app.route("/query", methods = ["GET"])
def pictures_exchange():
    cli_id = request.args["cli_id"]
    if cli_id not in clients:
        return '', 401 # Unauthorized
    query = parse.unquote(request.args["query"])
    tentatives = int(parse.unquote(request.args["tentatives"]))
    # Ask for query
    models = query_inference(query, tentatives)
    cache = cache_update(cache, query, cli_id, models, 60*10)

    return Response(response="HELLO PART 2", status=200)

    files = dict()
    for i, model in enumerate(models):
        filename = f"output_{hash(cli_id)}_{hash(query)}_{i}.gif"
        # If it appears there is already
        if os.path.exists(filename): os.remove(filename)

        export_to_gif((model), filename)
        files[f"gif_{i}.gif"] = open(filename, "rb")

    return jsonify(data = "files", status = 200, mimetype = "image/gif")



True
https://e410-34-105-92-156.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
